In [1]:
import itertools
from multiprocessing import Pool, cpu_count

import numpy as np
import seaborn as sns
from tqdm import tqdm

from package_file_conversion.nnlist2df import nnlist2df

In [2]:
def flatten_func(list_2dim):
    return list(itertools.chain.from_iterable(list_2dim))

In [3]:
def get_CO_bond_dist_list(nnlist_path='sample_test_files/POSCAR.nnlist'):
    df_nnlist = nnlist2df(nnlist_path)
    df_nnlist_C_O = df_nnlist[df_nnlist.apply(lambda row: (row['central_atom_symbol'] == 'N') and (row['neighboring_atom_symbol'] == 'H'), axis=1)]
    CO_bond_dist_list = df_nnlist_C_O['rel_distance'].tolist()
    return CO_bond_dist_list

In [5]:
# 元素種C, Oを含むPOSCARファイルパスリストをload
C_O_existed_poscar_folder_path_list = np.load('../get_some_speceis_existed_poscar_path_list/N_H_existed_poscar_folder_path_list.npy', allow_pickle=True)

In [6]:
C_O_existed_nnlist_5_path_list = [str(folder_p) + '/nnlist_5/POSCAR.nnlist' for folder_p in C_O_existed_poscar_folder_path_list]

In [7]:
# 並列化
try:
    p = Pool(cpu_count() - 1)
    CO_bond_dist_2d_list = list(tqdm(p.imap(get_CO_bond_dist_list, C_O_existed_nnlist_5_path_list[0:10000]), total=len(C_O_existed_nnlist_5_path_list)))
    # flatten
    CO_bond_dist_1d_list = flatten_func(CO_bond_dist_2d_list)
finally:
    p.close()
    p.join()

  5%|█████▎                                                                                                         | 10000/210889 [03:43<1:14:53, 44.71it/s]


In [8]:
np.save('NH_bond_dist_1d_list.npy', np.array(CO_bond_dist_1d_list))

#### メモ：ヒストグラムの描画は、別ファイルplot_CO_bond_dist_list.ipynbで行う